In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.decomposition import PCA #降维处理
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier


In [3]:
#载入数据
train_data = pd.read_csv('train.csv')

In [4]:
#Y特征数字化
Y=pd.Categorical(train_data['Attrition']).codes #codes表示数字化方法
Y.dtype

dtype('int8')

In [5]:
#删数据
X = train_data.drop(['user_id','Attrition','EmployeeCount','EmployeeNumber','StandardHours'],axis=1)

In [6]:
label_encode_list=['BusinessTravel','Department','EducationField','Gender',
                   'JobRole','MaritalStatus','OverTime','Over18']
newdf1 =pd.get_dummies(X[label_encode_list],prefix="label_") #特征数字化,加前缀
X = pd.concat([X,newdf1],axis=1)
X = X.drop(label_encode_list,axis=1)  #完成替换
X.head()


,Age,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,...,label__Research Director,label__Research Scientist,label__Sales Executive,label__Sales Representative,label__Divorced,label__Married,label__Single,label__No,label__Yes,label__Y
0,58,605,21,3,4,72,3,4,4,17875,...,0,0,0,0,0,1,0,0,1,1
1,45,950,28,3,4,97,3,1,4,2132,...,0,1,0,0,0,1,0,1,0,1
2,40,300,26,3,3,74,3,2,1,8396,...,0,0,1,0,0,1,0,1,0,1
3,36,1434,8,4,1,76,2,3,1,7587,...,0,0,1,0,0,0,1,1,0,1
4,25,599,24,1,3,73,1,1,4,1118,...,0,0,0,1,0,0,1,0,1,1


In [7]:
#age bins
def deal_age(row):
    age = row['Age']
    return int(age/10)


In [8]:
X['Age']=X.apply(deal_age,axis=1)   #对年龄分组

In [9]:
features = X.columns
scaler = StandardScaler()
scaler = scaler.fit_transform(X[features]) #z-score标准化
scaler = pd.DataFrame(scaler)


In [10]:
X=pd.concat([X,scaler],axis=1)
X=X.drop(X[features],axis=1)
X=X.values
train_x = X[:800,:]
train_y = Y[:800]
test_x = X[800:,:]
test_y =Y[800:]




In [11]:
#建立LR模型进行分类
mx  = LogisticRegression()
param_grid={'C':np.linspace(0.001,100,20)}  #给调参范围
#np.linspace:在指定间隔[0.001,100]内返回均匀间隔的数字，生成的样本数为20，可理解为下一步的迭代次数
grid_search=GridSearchCV(mx,param_grid,n_jobs=1,verbose=0)  #训练自动循环找最优的参数
#verbose 冗长度，决定是否输出训练过程
grid_search.fit(train_x,train_y)   #训练数据

best_parameters=grid_search.best_estimator_.get_params()    #寻找最优参数
mx1 = LogisticRegression(C=best_parameters['C'])    #选择最优参数
mx1.fit(train_x,train_y)     #训练
pred1=mx1.predict(test_x)
metrics.accuracy_score(pred1,test_y)


C:\Users\Ricardo\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Ricardo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Ricardo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Ricardo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Ricardo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be 

C:\Users\Ricardo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Ricardo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Ricardo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Ricardo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Ricardo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to

0.8776595744680851